## This is a set of modules for downloading historical equity data from Intrinio APIs and pushing it into an AWS RDS database.


First we import a variety of libraries and credentials to access APIs from Intrinio and AWS.

In [1]:
# Import credentials

import json
f = open("/. .<your file path here> . . /credentials.json")
credentials = json.load(f)

file_path = list(credentials.values())[0]
intrinio_key = list(credentials.values())[1]
aws_key = list(credentials.values())[2]
aws_secret_key = list(credentials.values())[3]
rds_host = list(credentials.values())[4]
rds_user = list(credentials.values())[5]
rds_password = list(credentials.values())[6]
rds_database = list(credentials.values())[7]
rds_charset = list(credentials.values())[8]


In [2]:
# Import Intrinio libraries

from __future__ import print_function
import time
import intrinio_sdk as intrinio
from intrinio_sdk.rest import ApiException

intrinio.ApiClient().configuration.api_key['api_key'] = intrinio_key

# Import the usual Python libraries

from tqdm.notebook import tqdm, trange  # to be used to track progress in loop iterations
import pandas as pd
import numpy as np

# Import Zip file libraries

from zipfile import ZipFile
from io import BytesIO
import urllib.request as urllib2

# Import the AWS libraries

import boto3
from boto3.s3.transfer import TransferConfig
from boto3.s3.transfer import S3Transfer
import io
import pyarrow as pa
import pyarrow.parquet as pq

# Declare the local file path to be used for saving CSV outputs.

global my_path
my_path = file_path



## Extract the historical data from Intrinio

Next, after manually downloading the historical bulk files from Intrinio, we roll them up into a single dataframe for transformation into the shape we need for the DB table. The source files are in a zip format, which is fine. The pd.read_csv command will unzip them automatically.

In [3]:
# Assemble the five Intrinio price history files into one dataframe.

def assemble_bulk_history():
    
    global df_price_history

    price_history = np.empty([0,29])

    for X in tqdm(range(1, 6)):

        ticker_file_path = my_path + "/" + "stock_prices_uscomp_all_file-" + str(X) + ".zip"
        data = pd.read_csv (ticker_file_path, low_memory = False)
        data = np.array(data.values)
        price_history = np.concatenate((price_history, data), axis=0)

    # Convert price history array to dataframe and do some cleanup

    df_price_history = pd.DataFrame(data = price_history, columns = ['security_id', 'company_id', 'name', 'cik', 'ticker', 'figi', 'composite_figi', 'composite_ticker', 'exchange_ticker', 'date', 'type', 'frequency', 'open', 'high', 'low', 'close', 'volume', 'adj_open', 'adj_high', 'adj_low', 'adj_close', 'adj_volume', 'adj_factor', 'ex_dividend',  'split_ratio', 'change', 'percent_change', 'fifty_two_week_high', 'fifty_two_week_low'])

    # Make sure Date column is in DateTime format, then sort by ticker and date

    df_price_history['date'] = pd.to_datetime(df_price_history['date'])
    df_price_history = df_price_history.sort_values(by=['ticker', 'date'])

    print("Price history files assembled.")    
    print("The shape of the price history dataframe is ", df_price_history.shape)
    
    return df_price_history


In [4]:
# Download the five Intrinio price history files and assemble them into one dataframe.

def download_bulk_history():
    
    global df_price_history

    price_history = np.empty([0,29])

    response = intrinio.BulkDownloadsApi().get_bulk_download_links()

    for X in tqdm(range(0, 5)):

        url = response.bulk_downloads[1].links[X].url
        r = urllib2.urlopen(url).read()
        file = ZipFile(BytesIO(r))
        data_csv = file.open("stock_prices_uscomp_all_file-" + str(X+1) + ".csv")
        data = pd.read_csv(data_csv, low_memory=False)
        data = np.array(data.values)
        price_history = np.concatenate((price_history, data), axis=0)

    # Convert price history array to dataframe and do some cleanup

    df_price_history = pd.DataFrame(data = price_history, columns = ['security_id', 'company_id', 'name', 'cik', 'ticker', 'figi', 'composite_figi', 'composite_ticker', 'exchange_ticker', 'date', 'type', 'frequency', 'open', 'high', 'low', 'close', 'volume', 'adj_open', 'adj_high', 'adj_low', 'adj_close', 'adj_volume', 'adj_factor', 'ex_dividend',  'split_ratio', 'change', 'percent_change', 'fifty_two_week_high', 'fifty_two_week_low'])

    # Make sure Date column is in DateTime format, then sort by ticker and date

    df_price_history['date'] = pd.to_datetime(df_price_history['date'])
    df_price_history = df_price_history.sort_values(by=['ticker', 'date'])

    print("Price history files assembled.")    
    print("The shape of the price history dataframe is ", df_price_history.shape)
    
    return df_price_history


In [5]:
# Get historical weighted average diluted shares outstanding for each ticker

def get_shares_out(myFigi, myTicker):
    
    global shares_out_list
    global shares_out_lists_combined
    
    identifier = myFigi
    tag = 'adjweightedavedilutedsharesos'
    frequency = ''
    type = ''
    start_date = ''
    end_date = ''
    sort_order = 'desc'
    page_size = 10000
    next_page = ''

    try:
        response = intrinio.HistoricalDataApi().get_historical_data(identifier, tag, frequency=frequency, type=type, start_date=start_date, end_date=end_date, sort_order=sort_order, page_size=page_size, next_page=next_page)
        shares_out_data = response.historical_data

        shares_out_list = []

        for item in range(len(shares_out_data)):
            
            # Append ticker and figi to results
            dict_item = shares_out_data[item].to_dict()
            dict_item['ticker'] = myTicker
            dict_item['figi'] = myFigi
            shares_out_list.append(dict_item)

    except:
        
        # Track tickers that do not have a shares out figure available.
        bad_tickers.append(myTicker, myFigi)
        pass

    shares_out_lists_combined.extend(shares_out_list)
    
    return shares_out_lists_combined


In [6]:
# Get the shares outstanding data and resample it to daily frequency.

def get_shares_out_data(df_price_history):

    import concurrent.futures

    mkt_cap_list = []
    shares_out_list = []
    bad_tickers = []
    shares_out_lists_combined = []

    # Grab tickers and figis from the price history DF and drop any figi duplicates that might show up.
    df_price_history_figis = df_price_history[['figi', 'ticker']].drop_duplicates(subset = ['figi'])
    arg_list = list(df_price_history_figis.to_records(index = False))

    # Use concurrent.futures to use multiple threads to retrieve shares out data.
    with concurrent.futures.ThreadPoolExecutor(max_workers = 25) as executor:
        executor.map(lambda f: get_shares_out(*f), arg_list)

    # Convert the shares out array to a dataframe, drop any duplicates, rename the values column, replace zeros with
    # NaNs, and get rid of any negative shares out numbers.
    df_shares_out = pd.DataFrame(shares_out_lists_combined)
    df_shares_out = df_shares_out.drop_duplicates(subset=['ticker', 'date'], keep = 'first')
    df_shares_out['date']= pd.to_datetime(df_shares_out['date'])
    df_shares_out = df_shares_out.rename(columns = {'value':'weighted_avg_shares_out'})
    df_shares_out['weighted_avg_shares_out'] = df_shares_out['weighted_avg_shares_out'].replace(0, np.nan)
    df_shares_out['weighted_avg_shares_out'] = df_shares_out['weighted_avg_shares_out'].abs()

    # set date as index and convert to daily periods. Since shares out are reported quarterly, we need to resample to
    # daily records.
    df_shares_resample = df_shares_out.copy()
    df_shares_resample = df_shares_resample.set_index('date')
    df_shares_resample.index = pd.to_datetime(df_shares_resample.index)
    df_shares_resample = df_shares_resample.groupby('ticker').resample('D', convention = 'end').ffill()
    df_shares_resample = df_shares_resample.droplevel('ticker')
    df_shares_resample = df_shares_resample.reset_index()

    df_shares_out = df_shares_resample.copy()

    # Take a look at the bad tickers that did not pull any results from the market cap query and make sure they are
    # not well recognized names/tickers. E.g. none should be MSFT or AAPL. For the most part, they should be ETFs or very
    # small cap stocks.

    df_bad_tickers = pd.DataFrame(bad_tickers, columns=['ticker'])
    df_bad_tickers = df_bad_tickers.drop_duplicates(keep = 'first')
    df_bad_tickers.to_csv(path_or_buf = my_path + "/df_bad_tickers.csv", index=False)

    print("The shape of the shares_out DF is ", df_shares_out.shape)
    print("Bad tickers that did not generate any shares out figures are pushed to CSV.")
    
    return df_shares_out


In [7]:
# Export the shares out history dataframe to a CSV file if you want to have a backup.
# df_shares_out.to_csv(path_or_buf = my_path + "/df_shares_out.csv", index=False)


In [8]:
# # In case the kernel fails later and we have to recreate the market cap history, we can grab the previous CSV file here instead.

# ticker_file_path = my_path + "/" + "df_shares_out.csv"
# df_shares_out = pd.read_csv (ticker_file_path, low_memory=False)
# df_shares_out.shape


## Finish transforming the data by adding a Market Cap calculation and several date fields to keep track of updates.

In [9]:
# Use left join to add shares out to history dataframe, then calculate market cap for each row.
# Make sure the date columns are in datetime format, do the merge, forward fill the shares out figures to from the
# last report date to today, then calculate market cap figures.

def join_dataframes(df_price_history, df_shares_out):
    
    global df_price_history_complete_load

    df_shares_out['date']= pd.to_datetime(df_shares_out['date'])
    df_price_history['date']= pd.to_datetime(df_price_history['date'])
    df_price_history_complete = pd.merge(df_price_history, df_shares_out, on=['ticker', 'figi', 'date'], how='left')
    df_price_history_complete = df_price_history_complete.sort_values(by = ['ticker', 'date'])
    df_price_history_complete['weighted_avg_shares_out'] = df_price_history_complete.groupby('ticker')['weighted_avg_shares_out'].transform(lambda x: x.ffill())
    df_price_history_complete['market_cap'] = df_price_history_complete['adj_close'] * df_price_history_complete['weighted_avg_shares_out']

    # # Export the complete price history dataframe to a CSV file.
    # df_price_history_complete.to_csv(path_or_buf = my_path + "/df_price_history_complete.csv", index=False)

    # Drop any remaining dupes and configure the final columns for loading into AWS RDS price history table.
    df_price_history_complete_load = df_price_history_complete.copy()
    df_price_history_complete_load = df_price_history_complete_load.dropna(subset = ['figi'])
    df_price_history_complete_load['date'] = pd.to_datetime(df_price_history_complete_load['date'])
    df_price_history_complete_load['intraperiod'] = 0
    df_price_history_complete_load['last_update_date'] = pd.to_datetime(df_price_history_complete_load['date'].max()).normalize()
    df_price_history_complete_load['last_corp_action_date'] = pd.to_datetime(df_price_history_complete_load['date'].max()).normalize()
    df_price_history_complete_load['key_id'] = df_price_history_complete_load['ticker'] + df_price_history_complete_load['figi'] + df_price_history_complete_load['date'].dt.strftime('%Y-%m-%d')
    df_price_history_complete_load = df_price_history_complete_load[['key_id', 'ticker', 'figi', 'date', 'open', 'high', 'low', 'close', 'volume',
           'adj_open', 'adj_high', 'adj_low', 'adj_close', 'adj_volume',
           'adj_factor', 'split_ratio', 'change', 'percent_change',
           'fifty_two_week_high', 'fifty_two_week_low', 'market_cap',
           'weighted_avg_shares_out', 'intraperiod', 'last_update_date', 'last_corp_action_date']]

    # Check primary key column again for duplicates.

    df_duplicate_rows = df_price_history_complete_load[df_price_history_complete_load.duplicated(subset = ['key_id'], keep = False)]
    if df_duplicate_rows.shape[0] != 0:
        df_duplicate_rows.to_csv(path_or_buf = my_path + "/df_duplicate_rows.csv", index=False)
        print("Primary key duplicates found and pushed to CSV file, df_duplicate_rows.csv.")

    print("Final price history dataframe completed, ready to load to DB. The final dataframe shape is ", df_price_history_complete_load.shape)

    return df_price_history_complete_load
    
                                                           

## For backup or archive purposes, save the final dataframe to CSV and/or parquet files and push them to AWS S3.

In [10]:
# Create the low level functional AWS client

def push_data_to_S3(df_price_history_complete_load):
    
    client = boto3.client(
        's3',
        aws_access_key_id = aws_key,
        aws_secret_access_key = aws_secret_key,
        region_name = 'us-east-1'
    )

    # Export the price history dataframe to a zipped CSV file then push to AWS S3.
    compression_opts = dict(method='zip', archive_name='df_full_price_history_complete_load.csv') 
    df_price_history_complete_load.to_csv(path_or_buf = my_path + "/df_full_price_history_complete_load.zip", index=False, compression=compression_opts)
    client.upload_file(my_path + "/df_full_price_history_complete_load.zip", 'bns-intrinio-data', "price-data-historical/csv_files/df_full_price_history_complete_load.zip")


    # Write parquet file to local drive, then push to AWS S3.
    local_file = my_path + "/df_full_price_history_complete_load.parquet"
    parquet_table = pa.Table.from_pandas(df_price_history)
    pq.write_table(parquet_table, local_file)
    client.upload_file(local_file, 'bns-intrinio-data', "price-data-historical/parquet_files/df_full_price_history_complete_load.parquet")

    print("Data saved to S3 in zipped CSV and parquet.")
    S3_push_status = "Done."
    
    return S3_push_status


In [11]:
# Upload the complete history file if needed

# file_path = my_path + "/" + "df_price_history_complete.zip"
# df_price_history_complete_load = pd.read_csv (file_path, low_memory=False)
# df_price_history_complete_load.shape

## Finally, create the MySQL price history table in RDS and push the history data into the table.

In [12]:
# Import SQL libraries

def create_and_fill_RDS_table(df_price_history_complete_load):

    import mysql.connector 
    from mysql.connector import errorcode

    from sqlalchemy import create_engine

    # Establish the MySQL connection

    connection = mysql.connector.connect(host=rds_host,
                                 user=rds_user, 
                                 password=rds_password, 
                                 database=rds_database,
                                 charset=rds_charset)

    mycursor = connection.cursor()

    # Create the data table in MySQL with MySQL Connector library

    create_price_data_history_table = """
    CREATE TABLE IF NOT EXISTS `price_data_historical` (
    `key_id` varchar(30) NOT NULL,
    `ticker` varchar(8) NOT NULL,
    `figi` varchar(14) NOT NULL,
    `date` datetime NOT NULL,
    `open` float NULL,
    `high` float NULL,
    `low` float NULL,
    `close` float NULL,
    `volume` float NULL,
    `adj_open` float NULL,
    `adj_high` float NULL,
    `adj_low` float NULL,
    `adj_close` float NULL,
    `adj_volume` float NULL,
    `split_ratio` float NULL,
    `adj_factor` float NULL,
    `change` float NULL,
    `percent_change` float NULL,
    `fifty_two_week_high` float NULL,
    `fifty_two_week_low` float NULL,
    `market_cap` bigint NULL,
    `weighted_avg_shares_out` bigint NULL,
    `intraperiod` boolean NOT NULL DEFAULT 0,
    `last_updated_date` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    `last_corp_action_date` timestamp NULL,
    PRIMARY KEY (key_id)
    ) ENGINE=InnoDB DEFAULT CHARSET=latin1;
    """

    mycursor.execute(create_price_data_history_table)

    connection.commit()

    print("The price_data_historical table is created in RDS.")


    # Push the final dataframe into SQL DB on AWS RDS.

    df = df_price_history_complete_load.copy()

    # Set SQLAlchemy database credentials.
    creds = {'usr': rds_user,
             'pwd': rds_password,
             'hst': rds_host,
             'prt': 3306,
             'dbn': rds_database}

    # MySQL conection string.
    connstr = 'mysql+mysqlconnector://{usr}:{pwd}@{hst}:{prt}/{dbn}'

    # Create sqlalchemy engine for MySQL connection.
    engine = create_engine(connstr.format(**creds))

    # Write DataFrame to MySQL using the engine (connection) created above.
    chunk = int(len(df) / 1000)
    df.to_sql(name='price_data_historical', 
                                          con=engine, 
                                          if_exists='replace', 
                                          chunksize=chunk,
                                          index=False)

    # Create indexes for ticker, figi and dates

    mycursor.execute("CREATE INDEX idx_ticker ON price_data_historical (ticker(9));")
    mycursor.execute("CREATE INDEX idx_figi ON price_data_historical (figi(12));")
    mycursor.execute("CREATE INDEX idx_date ON price_data_historical (date);")
    mycursor.execute("CREATE INDEX idx_update ON price_data_historical (last_update_date);")
    mycursor.execute("CREATE INDEX idx_corp_action ON price_data_historical (last_corp_action_date);")

    print("The price history data is loaded and the indexes are set.")
    rds_table_status = "Done."
    
    return rds_table_status


In [ ]:
# Run the ETL process.

assemble_bulk_history
download_bulk_history()
get_shares_out_data(df_price_history)
join_dataframes(df_price_history, df_shares_out)
push_data_to_S3(df_price_history_complete_load)
create_and_fill_RDS_table(df_price_history_complete_load)
